# Statistical Diagnostics and Model Validation
## Iusmorfos Framework - Advanced Statistical Testing

**Author**: Iusmorfos Research Team  
**Date**: 2024  
**Version**: 1.0

This notebook provides comprehensive statistical diagnostics and model validation for the Iusmorfos framework, including robustness testing, regression diagnostics, and power-law validation.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Bootstrap Robustness Testing](#bootstrap)
3. [Power-Law Distribution Validation](#power-law)
4. [Regression Diagnostics](#regression)
5. [Cross-Validation Analysis](#cross-validation)
6. [Sensitivity Analysis](#sensitivity)
7. [Outlier Detection and Impact](#outliers)
8. [Model Comparison and Selection](#model-comparison)
9. [Statistical Assumptions Testing](#assumptions)
10. [Reproducibility Validation](#reproducibility)

In [ ]:
# Environment setup
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path('../src').resolve()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import stats
from scipy.stats import powerlaw, pareto, norm, kstest, anderson
from scipy.optimize import minimize
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.stattools import durbin_watson
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

# Configuration
from config import get_config
from robustness import IusmorfosRobustnessAnalyzer

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("🔬 Statistical Diagnostics Notebook Initialized")
print(f"📊 Timestamp: {pd.Timestamp.now()}")

## 1. Setup and Configuration {#setup}

In [ ]:
# Initialize configuration and load data
config = get_config()
print(f"🔧 Configuration loaded: {config}")

# Initialize robustness analyzer
robustness_analyzer = IusmorfosRobustnessAnalyzer(n_bootstrap=1000)
print(f"🔬 Robustness analyzer initialized")

# Create or load sample data for statistical testing
np.random.seed(42)  # Ensure reproducibility

# Generate realistic legal innovation data for testing
n_samples = 842  # Argentina's documented innovations

# Complexity scores (1-10, beta distribution)
complexity_data = np.random.beta(2, 3, n_samples) * 9 + 1

# Citation counts (power-law distribution with γ ≈ 2.3)
citation_data = np.random.pareto(1.3, n_samples) * 2
citation_data = np.clip(citation_data, 0, 200).astype(int)

# Adoption success (0-1, beta distribution)
adoption_data = np.random.beta(3, 2, n_samples)

# Implementation time (months, lognormal)
implementation_time = np.random.lognormal(2.5, 0.8, n_samples)
implementation_time = np.clip(implementation_time, 1, 120)

# Years (temporal distribution)
years = np.random.choice(range(1990, 2024), n_samples)

# Create combined dataset
test_data = pd.DataFrame({
    'complexity': complexity_data,
    'citations': citation_data,
    'adoption_success': adoption_data,
    'implementation_months': implementation_time,
    'year': years
})

print(f"\n📊 Test Dataset Created:")
print(f"Shape: {test_data.shape}")
print(f"\nDescriptive Statistics:")
print(test_data.describe().round(3))

## 2. Bootstrap Robustness Testing {#bootstrap}

Test the robustness of key statistics using bootstrap resampling.

In [ ]:
print("🧪 BOOTSTRAP ROBUSTNESS TESTING")
print("=" * 40)

# Test complexity statistics
complexity_results = robustness_analyzer.test_complexity_evolution_robustness(
    test_data['complexity'].values
)

# Test citation network statistics
citation_results = robustness_analyzer.test_citation_network_robustness(
    test_data['citations'].values
)

# Test adoption success statistics
adoption_results = robustness_analyzer.test_adoption_success_robustness(
    test_data['adoption_success'].values
)

# Display results summary
print(f"\n📊 Bootstrap Results Summary:")
all_results = {**complexity_results, **citation_results, **adoption_results}

robust_tests = sum(1 for result in all_results.values() if result.robust)
total_tests = len(all_results)

print(f"Total tests: {total_tests}")
print(f"Robust tests: {robust_tests}")
print(f"Robustness rate: {robust_tests/total_tests:.1%}")

print(f"\n✅ Robust Statistics:")
for name, result in all_results.items():
    if result.robust:
        print(f"  • {result.test_name}: {result.original_value:.4f} ± {result.bootstrap_std:.4f}")

print(f"\n⚠️ Unstable Statistics:")
for name, result in all_results.items():
    if not result.robust:
        print(f"  • {result.test_name}: {result.original_value:.4f} (p={result.p_value:.4f})")

# Plot bootstrap distributions
robustness_analyzer.plot_bootstrap_distributions(save_plots=False)

## 3. Power-Law Distribution Validation {#power-law}

Validate the power-law hypothesis for citation distributions (γ=2.3).

In [ ]:
print("📊 POWER-LAW DISTRIBUTION VALIDATION")
print("=" * 45)

citations = test_data['citations'].values
citations_nonzero = citations[citations > 0]

print(f"Citation data: {len(citations)} total, {len(citations_nonzero)} non-zero")

# Create comprehensive power-law analysis plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Citation distribution histogram
axes[0, 0].hist(citations, bins=50, alpha=0.7, edgecolor='black', color='skyblue')
axes[0, 0].set_title('Citation Count Distribution')
axes[0, 0].set_xlabel('Citations')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_yscale('log')

# 2. Log-log plot for power-law fitting
if len(citations_nonzero) > 1:
    # Calculate frequency of each citation count
    unique_citations, counts = np.unique(citations_nonzero, return_counts=True)
    
    # Log-log plot
    axes[0, 1].loglog(unique_citations, counts, 'bo-', alpha=0.6, label='Data')
    
    # Fit power-law
    log_x = np.log(unique_citations)
    log_y = np.log(counts)
    
    if len(log_x) > 1:
        slope, intercept, r_value, p_value, std_err = stats.linregress(log_x, log_y)
        gamma_estimated = -slope
        
        # Plot fitted line
        x_fit = np.linspace(unique_citations.min(), unique_citations.max(), 100)
        y_fit = np.exp(intercept) * x_fit ** slope
        axes[0, 1].loglog(x_fit, y_fit, 'r--', alpha=0.8, 
                         label=f'γ = {gamma_estimated:.3f} (R² = {r_value**2:.3f})')
        
        axes[0, 1].set_title('Log-Log Plot (Power-Law Fit)')
        axes[0, 1].set_xlabel('Citation Count (log)')
        axes[0, 1].set_ylabel('Frequency (log)')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
        
        # Print power-law analysis results
        print(f"\n🔍 Power-Law Analysis Results:")
        print(f"Estimated γ: {gamma_estimated:.3f}")
        print(f"Expected γ (Iusmorfos): 2.3")
        print(f"Difference from expected: {abs(gamma_estimated - 2.3):.3f}")
        print(f"Goodness of fit (R²): {r_value**2:.3f}")
        print(f"Statistical significance (p): {p_value:.4f}")
        
        if abs(gamma_estimated - 2.3) < 0.5:
            print("✅ Close to expected γ=2.3")
        else:
            print("⚠️ Deviates from expected γ=2.3")

# 3. Cumulative distribution (CCDF)
sorted_citations = np.sort(citations_nonzero)
ccdf = 1 - np.arange(len(sorted_citations)) / len(sorted_citations)

axes[0, 2].loglog(sorted_citations, ccdf, 'go-', alpha=0.6, label='Empirical CCDF')

# Theoretical power-law CCDF
if 'gamma_estimated' in locals():
    x_min = sorted_citations[0]
    theoretical_ccdf = (sorted_citations / x_min) ** (-(gamma_estimated - 1))
    axes[0, 2].loglog(sorted_citations, theoretical_ccdf, 'r--', 
                     label=f'Power-law (γ={gamma_estimated:.2f})')

axes[0, 2].set_title('Cumulative Distribution (CCDF)')
axes[0, 2].set_xlabel('Citation Count (log)')
axes[0, 2].set_ylabel('P(X ≥ x) (log)')
axes[0, 2].legend()
axes[0, 2].grid(True)

# 4. Rank-frequency plot (Zipf's law)
ranks = np.arange(1, len(sorted_citations) + 1)
sorted_desc = sorted_citations[::-1]

axes[1, 0].loglog(ranks, sorted_desc, 'mo-', alpha=0.6, label='Rank-frequency')
axes[1, 0].set_title('Rank-Frequency Plot (Zipf\'s Law)')
axes[1, 0].set_xlabel('Rank (log)')
axes[1, 0].set_ylabel('Citation Count (log)')
axes[1, 0].grid(True)

# 5. Q-Q plot against theoretical power-law
if len(citations_nonzero) > 10:
    # Fit power-law using scipy
    try:
        fitted_params = powerlaw.fit(citations_nonzero)
        theoretical_quantiles = powerlaw.ppf(np.linspace(0.01, 0.99, len(citations_nonzero)), 
                                           *fitted_params)
        
        axes[1, 1].scatter(np.sort(theoretical_quantiles), np.sort(citations_nonzero), 
                          alpha=0.6, s=20)
        
        # Perfect fit line
        min_val = min(np.min(theoretical_quantiles), np.min(citations_nonzero))
        max_val = max(np.max(theoretical_quantiles), np.max(citations_nonzero))
        axes[1, 1].plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.8)
        
        axes[1, 1].set_title('Q-Q Plot: Power-Law')
        axes[1, 1].set_xlabel('Theoretical Quantiles')
        axes[1, 1].set_ylabel('Sample Quantiles')
        axes[1, 1].grid(True)
        
    except Exception as e:
        axes[1, 1].text(0.5, 0.5, f'Q-Q plot failed: {str(e)}', 
                        transform=axes[1, 1].transAxes, ha='center')

# 6. Goodness-of-fit tests
if len(citations_nonzero) > 10:
    # Kolmogorov-Smirnov test
    try:
        ks_stat, ks_p = kstest(citations_nonzero, lambda x: powerlaw.cdf(x, *fitted_params))
        
        # Anderson-Darling test (if available)
        try:
            ad_result = anderson(citations_nonzero, dist='norm')  # Note: limited distributions in scipy
            ad_stat = ad_result.statistic
        except:
            ad_stat = np.nan
        
        # Display test results
        test_results = f"""Goodness-of-Fit Tests:
        
Kolmogorov-Smirnov:
  Statistic: {ks_stat:.4f}
  p-value: {ks_p:.4f}
  {'✅ Fits power-law' if ks_p > 0.05 else '❌ Rejects power-law'}
  
Power-law Parameters:
  Shape (a): {fitted_params[0]:.3f}
  Location: {fitted_params[1]:.3f}
  Scale: {fitted_params[2]:.3f}
        """
        
        axes[1, 2].text(0.05, 0.95, test_results, transform=axes[1, 2].transAxes, 
                        fontsize=10, verticalalignment='top', fontfamily='monospace')
        axes[1, 2].set_title('Statistical Test Results')
        axes[1, 2].set_xticks([])
        axes[1, 2].set_yticks([])
        
        print(f"\n🧪 Goodness-of-Fit Tests:")
        print(f"Kolmogorov-Smirnov: D = {ks_stat:.4f}, p = {ks_p:.4f}")
        if ks_p > 0.05:
            print("✅ Data consistent with power-law distribution")
        else:
            print("❌ Data rejects power-law distribution")
            
    except Exception as e:
        axes[1, 2].text(0.5, 0.5, f'Statistical tests failed: {str(e)}', 
                        transform=axes[1, 2].transAxes, ha='center')

plt.tight_layout()
plt.show()

# Network concentration analysis (Pareto principle)
total_citations = np.sum(citations)
sorted_desc_all = np.sort(citations)[::-1]
cumsum_citations = np.cumsum(sorted_desc_all)

# Find 80% threshold
threshold_80 = 0.8 * total_citations
pareto_80_index = np.where(cumsum_citations >= threshold_80)[0]

if len(pareto_80_index) > 0:
    pareto_80_pct = (pareto_80_index[0] + 1) / len(citations) * 100
    print(f"\n📊 Network Concentration Analysis:")
    print(f"80/20 Rule: Top {pareto_80_pct:.1f}% of innovations account for 80% of citations")
    
    if pareto_80_pct <= 25:
        print("✅ Strong Pareto effect (heavy concentration)")
    elif pareto_80_pct <= 40:
        print("📊 Moderate concentration")
    else:
        print("📈 Weak concentration")

## 4. Regression Diagnostics {#regression}

Perform comprehensive regression diagnostics for model relationships.

In [ ]:
print("📈 REGRESSION DIAGNOSTICS")
print("=" * 30)

# Create regression model: adoption_success ~ complexity + log(citations + 1) + implementation_time
# Prepare data
y = test_data['adoption_success'].values
X_raw = test_data[['complexity', 'citations', 'implementation_months']].copy()
X_raw['log_citations'] = np.log1p(X_raw['citations'])  # log(citations + 1)
X_raw['log_implementation'] = np.log(X_raw['implementation_months'])

# Select final features
feature_cols = ['complexity', 'log_citations', 'log_implementation']
X = X_raw[feature_cols].values

# Add intercept for statsmodels
X_sm = sm.add_constant(X)

# Fit OLS model
model = sm.OLS(y, X_sm).fit()

print(f"\n📊 Regression Model Results:")
print(model.summary())

# Extract predictions and residuals
y_pred = model.fittedvalues
residuals = model.resid
standardized_residuals = residuals / np.std(residuals)

# Create diagnostic plots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Residuals vs Fitted
axes[0, 0].scatter(y_pred, residuals, alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Fitted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Fitted')
axes[0, 0].grid(True, alpha=0.3)

# Add LOWESS smoother
from statsmodels.nonparametric.smoothers_lowess import lowess
lowess_result = lowess(residuals, y_pred, frac=0.3)
axes[0, 0].plot(lowess_result[:, 0], lowess_result[:, 1], 'orange', linewidth=2, label='LOWESS')
axes[0, 0].legend()

# 2. Q-Q plot of residuals
sm.qqplot(standardized_residuals, line='45', ax=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot of Residuals')
axes[0, 1].grid(True, alpha=0.3)

# 3. Scale-Location plot
sqrt_abs_resid = np.sqrt(np.abs(standardized_residuals))
axes[0, 2].scatter(y_pred, sqrt_abs_resid, alpha=0.6)
axes[0, 2].set_xlabel('Fitted Values')
axes[0, 2].set_ylabel('√|Standardized Residuals|')
axes[0, 2].set_title('Scale-Location Plot')
axes[0, 2].grid(True, alpha=0.3)

# LOWESS for scale-location
lowess_sl = lowess(sqrt_abs_resid, y_pred, frac=0.3)
axes[0, 2].plot(lowess_sl[:, 0], lowess_sl[:, 1], 'orange', linewidth=2)

# 4. Residuals histogram
axes[1, 0].hist(standardized_residuals, bins=30, alpha=0.7, density=True, edgecolor='black')
axes[1, 0].set_xlabel('Standardized Residuals')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Histogram of Residuals')

# Overlay normal distribution
x_norm = np.linspace(standardized_residuals.min(), standardized_residuals.max(), 100)
y_norm = stats.norm.pdf(x_norm, 0, 1)
axes[1, 0].plot(x_norm, y_norm, 'r-', linewidth=2, label='Normal(0,1)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 5. Leverage plot (Cook's distance)
influence = model.get_influence()
leverage = influence.hat_matrix_diag
cooks_d = influence.cooks_distance[0]

axes[1, 1].scatter(leverage, standardized_residuals, alpha=0.6, c=cooks_d, 
                  cmap='viridis', s=30)
axes[1, 1].set_xlabel('Leverage')
axes[1, 1].set_ylabel('Standardized Residuals')
axes[1, 1].set_title('Residuals vs Leverage')
axes[1, 1].grid(True, alpha=0.3)

# Add Cook's distance contours
p = X.shape[1] + 1  # number of parameters
n = len(y)
cooks_threshold = 4 / n
axes[1, 1].axhline(y=2, color='r', linestyle='--', alpha=0.7, label='|res| = 2')
axes[1, 1].axhline(y=-2, color='r', linestyle='--', alpha=0.7)
axes[1, 1].legend()

# 6. Actual vs Predicted
axes[1, 2].scatter(y, y_pred, alpha=0.6)
axes[1, 2].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', linewidth=2)
axes[1, 2].set_xlabel('Actual Values')
axes[1, 2].set_ylabel('Predicted Values')
axes[1, 2].set_title('Actual vs Predicted')
axes[1, 2].grid(True, alpha=0.3)

# Add R² to the plot
r2 = r2_score(y, y_pred)
axes[1, 2].text(0.05, 0.95, f'R² = {r2:.3f}', transform=axes[1, 2].transAxes, 
               fontsize=12, verticalalignment='top', 
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Statistical tests for assumptions
print(f"\n🧪 REGRESSION ASSUMPTION TESTS:")
print("=" * 40)

# 1. Normality of residuals
shapiro_stat, shapiro_p = stats.shapiro(standardized_residuals)
jarque_stat, jarque_p = stats.jarque_bera(residuals)

print(f"\n1. Normality of Residuals:")
print(f"   Shapiro-Wilk: W = {shapiro_stat:.4f}, p = {shapiro_p:.4f}")
print(f"   Jarque-Bera: JB = {jarque_stat:.4f}, p = {jarque_p:.4f}")
if shapiro_p > 0.05 and jarque_p > 0.05:
    print("   ✅ Residuals appear normally distributed")
else:
    print("   ⚠️ Residuals may not be normally distributed")

# 2. Homoscedasticity (constant variance)
bp_stat, bp_p, _, _ = het_breuschpagan(residuals, X_sm)
white_stat, white_p, _, _ = het_white(residuals, X_sm)

print(f"\n2. Homoscedasticity:")
print(f"   Breusch-Pagan: LM = {bp_stat:.4f}, p = {bp_p:.4f}")
print(f"   White test: LM = {white_stat:.4f}, p = {white_p:.4f}")
if bp_p > 0.05 and white_p > 0.05:
    print("   ✅ Homoscedasticity assumption satisfied")
else:
    print("   ⚠️ Heteroscedasticity detected")

# 3. Independence (Durbin-Watson test)
dw_stat = durbin_watson(residuals)
print(f"\n3. Independence:")
print(f"   Durbin-Watson: {dw_stat:.4f}")
if 1.5 < dw_stat < 2.5:
    print("   ✅ No evidence of autocorrelation")
else:
    print("   ⚠️ Possible autocorrelation in residuals")

# 4. Multicollinearity (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

print(f"\n4. Multicollinearity (VIF):")
for i in range(1, X_sm.shape[1]):  # Skip intercept
    vif = variance_inflation_factor(X_sm, i)
    feature_name = ['Complexity', 'Log Citations', 'Log Implementation'][i-1]
    print(f"   {feature_name}: VIF = {vif:.2f}")
    
max_vif = max([variance_inflation_factor(X_sm, i) for i in range(1, X_sm.shape[1])])
if max_vif < 5:
    print(f"   ✅ Low multicollinearity (max VIF = {max_vif:.2f})")
elif max_vif < 10:
    print(f"   ⚠️ Moderate multicollinearity (max VIF = {max_vif:.2f})")
else:
    print(f"   ❌ High multicollinearity (max VIF = {max_vif:.2f})")

# 5. Outliers and influential points
high_leverage = leverage > (2 * p / n)  # Rule of thumb: 2p/n
high_cooks = cooks_d > cooks_threshold
outlier_residuals = np.abs(standardized_residuals) > 3

print(f"\n5. Outliers and Influential Points:")
print(f"   High leverage points: {np.sum(high_leverage)} ({np.sum(high_leverage)/n:.1%})")
print(f"   High Cook's distance: {np.sum(high_cooks)} ({np.sum(high_cooks)/n:.1%})")
print(f"   Outlier residuals (|z| > 3): {np.sum(outlier_residuals)} ({np.sum(outlier_residuals)/n:.1%})")

if np.sum(high_cooks) < 0.05 * n:
    print("   ✅ Few influential points detected")
else:
    print("   ⚠️ Multiple influential points detected")

# Model performance summary
print(f"\n📊 MODEL PERFORMANCE SUMMARY:")
print(f"R-squared: {model.rsquared:.4f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"F-statistic: {model.fvalue:.4f} (p = {model.f_pvalue:.4f})")
print(f"AIC: {model.aic:.2f}")
print(f"BIC: {model.bic:.2f}")
print(f"Root MSE: {np.sqrt(model.mse_resid):.4f}")

## 5. Cross-Validation Analysis {#cross-validation}

Perform cross-validation to assess model stability and generalizability.

In [ ]:
print("🔄 CROSS-VALIDATION ANALYSIS")
print("=" * 35)

from sklearn.model_selection import KFold, cross_validate, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Prepare data for scikit-learn
X_cv = X  # Features without intercept for sklearn
y_cv = y

# Initialize model
lr_model = LinearRegression()

# Define cross-validation strategy
cv_folds = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation with multiple metrics
cv_scores = cross_validate(
    lr_model, X_cv, y_cv, cv=cv_folds,
    scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],
    return_train_score=True
)

# Calculate additional metrics
test_r2 = cv_scores['test_r2']
train_r2 = cv_scores['train_r2']
test_mse = -cv_scores['test_neg_mean_squared_error']
train_mse = -cv_scores['train_neg_mean_squared_error']
test_mae = -cv_scores['test_neg_mean_absolute_error']
train_mae = -cv_scores['train_neg_mean_absolute_error']

# Display cross-validation results
print(f"\n📊 Cross-Validation Results (5-fold):")
print(f"\nR² Score:")
print(f"  Training:   {train_r2.mean():.4f} ± {train_r2.std():.4f}")
print(f"  Test:       {test_r2.mean():.4f} ± {test_r2.std():.4f}")
print(f"  Difference: {(train_r2.mean() - test_r2.mean()):.4f}")

print(f"\nMean Squared Error:")
print(f"  Training:   {train_mse.mean():.6f} ± {train_mse.std():.6f}")
print(f"  Test:       {test_mse.mean():.6f} ± {test_mse.std():.6f}")

print(f"\nMean Absolute Error:")
print(f"  Training:   {train_mae.mean():.6f} ± {train_mae.std():.6f}")
print(f"  Test:       {test_mae.mean():.6f} ± {test_mae.std():.6f}")

# Overfitting assessment
r2_gap = train_r2.mean() - test_r2.mean()
if r2_gap < 0.05:
    overfitting_status = "✅ No significant overfitting"
elif r2_gap < 0.1:
    overfitting_status = "⚠️ Mild overfitting"
else:
    overfitting_status = "❌ Significant overfitting"

print(f"\nOverfitting Assessment: {overfitting_status} (gap = {r2_gap:.4f})")

# Cross-validation stability
cv_stability = test_r2.std() / test_r2.mean() if test_r2.mean() != 0 else np.inf
if cv_stability < 0.1:
    stability_status = "✅ High stability"
elif cv_stability < 0.2:
    stability_status = "📊 Moderate stability"
else:
    stability_status = "⚠️ Low stability"

print(f"Cross-validation stability: {stability_status} (CV = {cv_stability:.3f})")

# Create cross-validation visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. R² scores across folds
folds = range(1, 6)
axes[0, 0].plot(folds, train_r2, 'bo-', label='Training', linewidth=2, markersize=8)
axes[0, 0].plot(folds, test_r2, 'ro-', label='Validation', linewidth=2, markersize=8)
axes[0, 0].fill_between(folds, train_r2, alpha=0.3, color='blue')
axes[0, 0].fill_between(folds, test_r2, alpha=0.3, color='red')
axes[0, 0].set_xlabel('Fold')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].set_title('R² Scores Across Folds')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. MSE across folds
axes[0, 1].plot(folds, train_mse, 'bo-', label='Training', linewidth=2, markersize=8)
axes[0, 1].plot(folds, test_mse, 'ro-', label='Validation', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Fold')
axes[0, 1].set_ylabel('Mean Squared Error')
axes[0, 1].set_title('MSE Across Folds')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Learning curves
train_sizes = np.linspace(0.1, 1.0, 10)
train_sizes_abs, train_scores_lc, val_scores_lc = learning_curve(
    lr_model, X_cv, y_cv, cv=5, train_sizes=train_sizes, 
    scoring='r2', random_state=42
)

train_mean = train_scores_lc.mean(axis=1)
train_std = train_scores_lc.std(axis=1)
val_mean = val_scores_lc.mean(axis=1)
val_std = val_scores_lc.std(axis=1)

axes[1, 0].plot(train_sizes_abs, train_mean, 'bo-', label='Training Score')
axes[1, 0].fill_between(train_sizes_abs, train_mean - train_std, 
                       train_mean + train_std, alpha=0.3, color='blue')
axes[1, 0].plot(train_sizes_abs, val_mean, 'ro-', label='Validation Score')
axes[1, 0].fill_between(train_sizes_abs, val_mean - val_std, 
                       val_mean + val_std, alpha=0.3, color='red')
axes[1, 0].set_xlabel('Training Set Size')
axes[1, 0].set_ylabel('R² Score')
axes[1, 0].set_title('Learning Curves')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Prediction intervals across folds
all_predictions = []
all_actuals = []

for train_idx, test_idx in cv_folds.split(X_cv):
    X_train_fold, X_test_fold = X_cv[train_idx], X_cv[test_idx]
    y_train_fold, y_test_fold = y_cv[train_idx], y_cv[test_idx]
    
    fold_model = LinearRegression().fit(X_train_fold, y_train_fold)
    y_pred_fold = fold_model.predict(X_test_fold)
    
    all_predictions.extend(y_pred_fold)
    all_actuals.extend(y_test_fold)

all_predictions = np.array(all_predictions)
all_actuals = np.array(all_actuals)

axes[1, 1].scatter(all_actuals, all_predictions, alpha=0.6, s=30)
axes[1, 1].plot([all_actuals.min(), all_actuals.max()], 
               [all_actuals.min(), all_actuals.max()], 'r--', linewidth=2)
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].set_title('Cross-Validated Predictions')

# Calculate and display overall CV R²
cv_r2_overall = r2_score(all_actuals, all_predictions)
axes[1, 1].text(0.05, 0.95, f'CV R² = {cv_r2_overall:.3f}', 
               transform=axes[1, 1].transAxes, fontsize=12, 
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ Cross-validation analysis complete")
print(f"Overall CV R²: {cv_r2_overall:.4f}")
print(f"Model appears {'stable' if cv_stability < 0.15 else 'unstable'} across different data splits")

## 6. Reproducibility Validation {#reproducibility}

Verify that results are reproducible with the same random seed.

In [ ]:
print("🔄 REPRODUCIBILITY VALIDATION")
print("=" * 35)

# Function to run complete analysis
def run_analysis_with_seed(seed):
    """Run complete analysis with given seed."""
    np.random.seed(seed)
    
    # Generate data
    n = 842
    complexity = np.random.beta(2, 3, n) * 9 + 1
    citations = np.random.pareto(1.3, n) * 2
    citations = np.clip(citations, 0, 200).astype(int)
    adoption = np.random.beta(3, 2, n)
    
    # Calculate key statistics
    results = {
        'complexity_mean': np.mean(complexity),
        'complexity_std': np.std(complexity),
        'citations_mean': np.mean(citations),
        'citations_max': np.max(citations),
        'adoption_mean': np.mean(adoption),
        'adoption_std': np.std(adoption)
    }
    
    # Power-law analysis
    citations_nz = citations[citations > 0]
    if len(citations_nz) > 1:
        unique_cit, counts = np.unique(citations_nz, return_counts=True)
        if len(unique_cit) > 1:
            log_x = np.log(unique_cit)
            log_y = np.log(counts)
            slope, intercept, r_val, p_val, std_err = stats.linregress(log_x, log_y)
            results['power_law_gamma'] = -slope
            results['power_law_r2'] = r_val**2
        else:
            results['power_law_gamma'] = 0
            results['power_law_r2'] = 0
    else:
        results['power_law_gamma'] = 0
        results['power_law_r2'] = 0
    
    return results

# Test reproducibility with same seed
print("🧪 Testing reproducibility with seed=42...")

results_1 = run_analysis_with_seed(42)
results_2 = run_analysis_with_seed(42)
results_3 = run_analysis_with_seed(42)

# Check if results are identical
reproducible = True
tolerance = 1e-10

print(f"\n📊 Reproducibility Test Results:")
for key in results_1.keys():
    val_1 = results_1[key]
    val_2 = results_2[key]
    val_3 = results_3[key]
    
    diff_12 = abs(val_1 - val_2)
    diff_13 = abs(val_1 - val_3)
    diff_23 = abs(val_2 - val_3)
    
    max_diff = max(diff_12, diff_13, diff_23)
    
    if max_diff <= tolerance:
        status = "✅ IDENTICAL"
    else:
        status = "❌ DIFFERENT"
        reproducible = False
    
    print(f"{key:20s}: {val_1:10.6f} {status} (max diff: {max_diff:.2e})")

print(f"\n🎯 Overall Reproducibility: {'✅ PASSED' if reproducible else '❌ FAILED'}")

# Test with different seeds to show variability
print(f"\n🔀 Testing variability with different seeds...")

seeds = [42, 123, 456, 789, 999]
seed_results = []

for seed in seeds:
    result = run_analysis_with_seed(seed)
    seed_results.append(result)

# Calculate variability across seeds
variability_df = pd.DataFrame(seed_results, index=[f'Seed_{s}' for s in seeds])

print(f"\n📈 Variability Across Seeds:")
print(variability_df.round(4))

# Calculate coefficient of variation for each statistic
print(f"\n📊 Coefficient of Variation (CV) Across Seeds:")
for col in variability_df.columns:
    mean_val = variability_df[col].mean()
    std_val = variability_df[col].std()
    cv = std_val / mean_val if mean_val != 0 else np.inf
    print(f"{col:20s}: CV = {cv:8.4f} ({'Low' if cv < 0.1 else 'Medium' if cv < 0.3 else 'High'} variability)")

# Visualize seed stability
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

key_stats = ['complexity_mean', 'citations_mean', 'adoption_mean', 
            'power_law_gamma', 'complexity_std', 'adoption_std']

for i, stat in enumerate(key_stats):
    if i < len(axes) and stat in variability_df.columns:
        values = variability_df[stat].values
        
        axes[i].bar(range(len(seeds)), values, alpha=0.7, 
                   color=plt.cm.viridis(i/len(key_stats)))
        axes[i].set_title(f'{stat.replace("_", " ").title()}')
        axes[i].set_xlabel('Seed')
        axes[i].set_ylabel('Value')
        axes[i].set_xticks(range(len(seeds)))
        axes[i].set_xticklabels(seeds, rotation=45)
        axes[i].grid(True, alpha=0.3)
        
        # Add mean line
        mean_line = np.mean(values)
        axes[i].axhline(y=mean_line, color='red', linestyle='--', alpha=0.8, 
                       label=f'Mean: {mean_line:.3f}')
        axes[i].legend()

plt.tight_layout()
plt.show()

# Generate reproducibility report
reproducibility_report = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'reproducibility_passed': reproducible,
    'tolerance_used': tolerance,
    'seeds_tested': seeds,
    'identical_results_with_same_seed': reproducible,
    'variability_across_seeds': variability_df.std().to_dict(),
    'mean_values': variability_df.mean().to_dict(),
    'recommendations': [
        "Always use fixed random seeds for reproducible research",
        "Report random seeds used in publications",
        "Test robustness across multiple seeds",
        "Document any sources of non-determinism"
    ]
}

# Save reproducibility report
report_path = config.get_path('results_dir') / f"reproducibility_report_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_path, 'w') as f:
    json.dump(reproducibility_report, f, indent=2, default=str)

print(f"\n💾 Reproducibility report saved: {report_path}")
print(f"\n✅ Statistical diagnostics notebook complete!")